DNN과 다르게 RNN에서는 okt형태소 토큰화 후 워드임베딩을 통해 문장의 문맥을 파악하는 방식으로 학습 시킬것이다. 
일반적으로 불용어를 제거=> 노이즈를 줄여서 성능향상을 도모
그러나 감성분류 5가지에선 불용어 제거하지않고 형태소를 분리하고 어절을 그대로 사용한다.

In [14]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
train_text = open(train_file,'rb').read().decode(encoding='utf-8')


In [15]:
import pandas as pd

df_train = pd.DataFrame({
    'feature':[row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0],
    'target': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})


In [16]:
#데이터 크기 조절
df_train= df_train[::4]

In [17]:
#문장 추출
texts= [ ]
for i in df_train['feature']:
    texts.append(i)
len(texts)

37500

In [18]:
#문자열이 아닌게 있는지 확인
for i in texts:
    if type(i)!=str:
        print(i)

In [19]:
# train 데이터의 입력(X)에 대한 정제(Cleaning)
import re
from soynlp.normalizer import repeat_normalize

def clean_korean_text(text):
    # 특수 문자 및 숫자 제거
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)
    # 반복되는 자음, 모음 제거 (e.g., 'ㅋㅋㅋ' -> 'ㅋ')
    text = repeat_normalize(text, num_repeats=1)
    # 띄어쓰기 정규화 (연속된 공백 문자를 하나의 공백 문자로 변환)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

clean_texts=[]
for i in texts:
    clean_texts.append(clean_korean_text(i))

In [20]:
len(clean_texts)

37500

In [21]:
#Mecab 토큰화
from konlpy.tag import Mecab
mecab=Mecab(dicpath=r'C:/mecab/mecab-ko-dic')

tokenized_text = [ mecab.morphs(i) for i in clean_texts]

In [22]:
len(tokenized_text)
tokenized_text

[['아', '더', '빙', '진짜', '짜증', '나', '네요', '목소리'],
 ['사이몬페그',
  '의',
  '익살',
  '스런',
  '연기',
  '가',
  '돋보였',
  '던',
  '영화',
  '스파이더맨',
  '에서',
  '늙',
  '어',
  '보이',
  '기',
  '만',
  '했',
  '던',
  '커스틴',
  '던스트',
  '가',
  '너무나',
  '도',
  '이뻐',
  '보였',
  '다'],
 ['액션', '이', '없', '는데', '도', '재미', '있', '는', '몇', '안', '되', '는', '영화'],
 ['울',
  '면서',
  '손들',
  '고',
  '횡단',
  '보도',
  '건널',
  '때',
  '뛰쳐나올',
  '뻔',
  '이범수',
  '연기',
  '드럽',
  '게',
  '못해'],
 ['참',
  '사람',
  '들',
  '웃긴',
  '게',
  '바스코',
  '가',
  '이기면',
  '락스',
  '코',
  '라고',
  '까',
  '고',
  '바비',
  '가',
  '이기',
  '면',
  '아이돌',
  '이',
  '라고',
  '깐다',
  '그냥',
  '까',
  '고',
  '싶',
  '어서',
  '안달',
  '난',
  '것',
  '처럼',
  '보인다'],
 ['나름',
  '심오',
  '한',
  '뜻',
  '도',
  '있',
  '는',
  '듯',
  '그냥',
  '학생',
  '이',
  '선생',
  '과',
  '놀아나',
  '는',
  '영화',
  '는',
  '절대',
  '아님'],
 ['주제', '는', '좋', '은데', '중반', '부터', '지루', '하', '다'],
 ['재', '밋', '는', '뎅'],
 ['재밌', '는데', '별점', '이', '왜', '이리', '낮', '은고'],
 ['키이라',
  '나이틀리',
  '가',
  '연기',
  '하',
  '고자',

In [23]:
#데이터 불균형 확인
print(df_train.iloc[:,1][df_train.iloc[:,1]==0].count())
print(df_train.iloc[:,1][df_train.iloc[:,1]==1].count())

18678
18822


In [24]:
#y값 추출, 데이터 차원 맞추기, 넘파이 배열로 변환
y_train=(df_train.iloc[:,1].to_frame()).to_numpy()
y_train

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [1]], dtype=int64)

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#tokenized_text의 빈도수 상위 만개 어절을 토크나이저에 fit시킴
tokenizer = Tokenizer(num_words=3000)
tokenizer.fit_on_texts(tokenized_text)

#단어 빈도수 시각화 => 몇개를 기준으로하면 좋을지
#불용어제거후 시각화해볼수도있음

#시퀀스데이터로 변환 ==벡터화
sequences = tokenizer.texts_to_sequences(tokenized_text)

In [26]:
#길이를 맞춰주는 패딩작업,
x_train = pad_sequences(sequences, maxlen=40, padding='post')

In [27]:
#시퀀스 길이확인
print(x_train[0])
x_train.shape

[ 35  80 943  41 245  22  40 646   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]


(37500, 40)

In [28]:
# GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding,BatchNormalization,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Embedding( 3000, 300, input_length=40 ),
    GRU(32),
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
optimizer= Adam(learning_rate =0.002)
model.compile(optimizer=optimizer , loss='binary_crossentropy', metrics=['accuracy'])

#콜백정의
ES =EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)

#모델학습
model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.25, callbacks=[ES])


Epoch 1/20
220/220 [==============================] - 23s 78ms/step - loss: 0.6971 - accuracy: 0.5009 - val_loss: 0.6927 - val_accuracy: 0.5037
Epoch 2/20
220/220 [==============================] - 17s 76ms/step - loss: 0.5631 - accuracy: 0.6789 - val_loss: 0.4147 - val_accuracy: 0.8055
Epoch 3/20
220/220 [==============================] - 16s 71ms/step - loss: 0.3854 - accuracy: 0.8341 - val_loss: 0.4036 - val_accuracy: 0.8196
Epoch 4/20
220/220 [==============================] - 15s 69ms/step - loss: 0.3313 - accuracy: 0.8620 - val_loss: 0.4211 - val_accuracy: 0.8161
Epoch 5/20
220/220 [==============================] - 16s 71ms/step - loss: 0.2976 - accuracy: 0.8791 - val_loss: 0.4321 - val_accuracy: 0.8176
Epoch 6/20
220/220 [==============================] - 15s 67ms/step - loss: 0.2622 - accuracy: 0.8956 - val_loss: 0.5140 - val_accuracy: 0.8063
Epoch 7/20
220/220 [==============================] - 15s 67ms/step - loss: 0.2290 - accuracy: 0.9094 - val_loss: 0.5228 - val_accuracy:

In [29]:
#테스트문장 입력
test = ['회식끝나고 집가는중','개꿀잼이네','진짜 너무 별로다.','ㅋㅋㅋㅋㅋㅋㅋㅋ']

#정제
clean_test_texts=[]
for i in test:
    clean_test_texts.append(clean_korean_text(i))

#mecab
tokenized_test_text = [ mecab.morphs(i) for i in clean_test_texts]

#벡터화
test_sequences = tokenizer.texts_to_sequences(tokenized_test_text) 
x_test = pad_sequences(test_sequences,padding='post',maxlen=40) 

#예측
prediction = model.predict(x_test)  

for i in np.round(prediction):
    print(int(i))

1/1 [==============================] - 1s 749ms/step
0
1
0
0


# 카톡대화 입력

In [30]:
# 카톡대화 불러와서 정제,(정규화코드)하는 함수
import re

def get_from_txt(txt):
    data= open(txt,"r", encoding='utf-8').read().split('\n')
    sentences=[]
    for line in data:
        pattern = r'\[(.*?)\]\s+\[(.*?)\]\s+(.+)'
        match = re.match(pattern, line)
        if match:
            name = match.group(1)  # 첫 번째 대괄호 안의 단어 추출
            time = match.group(2)  # 두 번째 대괄호 안의 단어 추출
            content = match.group(3)  # 대괄호 뒤의 내용 추출
            # print(name, time, content)
            temp=[name,time,content]
            sentences.append(temp)    
    return sentences

In [31]:
#닉네임 입력단 
target_name = str(input())
print(target_name)  #김하영 입력

김하영


In [32]:
#입력된 이름의 '대화내역만' 담기
received_texts= []
for i in get_from_txt('sample.txt'): 
    if i[0] == target_name:
        received_texts.append( i[2] )
        
# 이모티콘, 사진, 샵검색 제거 
clean_received_texts1 = []
for i in received_texts:
    if '샵검색:' not in i: 
        if "이모티콘" not in i:
            if '샵검색:' not in i:
                clean_received_texts1.append(str(i))

#=========== 이쯤에서 답장시간 계산기 구현   =======================
          
          
#텍스트 정제  
clean_received_texts2= [clean_korean_text(i) for i in clean_received_texts1]


#문자열이 아닌게 있는지 확인
for i in clean_received_texts2:
    if type(i)!=str:
        print(i)
        
# train 데이터 입력값(X)을 정제(Cleaning)
import re
from soynlp.normalizer import repeat_normalize

def clean_korean_text(text):
    # 특수 문자 및 숫자 제거
    text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)
    # 반복되는 자음, 모음 제거 (e.g., 'ㅋㅋㅋ' -> 'ㅋ')
    text = repeat_normalize(text, num_repeats=1)
    # 띄어쓰기 정규화 (연속된 공백 문자를 하나의 공백 문자로 변환)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


clean_test_texts=[]
for i in clean_received_texts2:
    clean_test_texts.append(clean_korean_text(i))


#mecab으로 토큰화
tokenized_clean_test_texts= [mecab.morphs(i) for i in clean_test_texts ]


#시퀀스데이터로 변환
test_sequences = tokenizer.texts_to_sequences(tokenized_clean_test_texts) 
paded_test_sequences = pad_sequences(test_sequences,padding='post',maxlen=40) 
prediction = model.predict(paded_test_sequences)  


cnt0=0;cnt1=0

for i in prediction:
    if float(i)>=0.5:
        print('긍정문입니다.')
        cnt1+=1
    elif float(i)<0.5:
        print('부정문입니다.')
        cnt0+=1
        
#부정과 긍정문의 갯수
print('부정문과 긍정문의 갯수:',cnt0,cnt1)

#긍정과 부정의 비율( 긍정문의 수 / 부정문의 수)
print('긍정과 부정의 비율: ',cnt1/cnt0)
#숫자가 1이상이고 높을수록 긍정적이다.

6/6 [==============================] - 0s 11ms/step
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
긍정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입니다.
부정문입